In [1]:
!pip install yolov5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# import sys
# sys.path.append("yolov5")

# from yolov5.models.yolo import Model, ComputeLoss

In [22]:
# Include all packages
import os
import cv2
import time
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from yolov5.models.yolo import Model
from yolov5.utils.loss import ComputeLoss
from sklearn.model_selection import train_test_split



In [5]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/DL/FinalProject/signDatabasePublicFramesOnly.zip', 'r') as zip_ref:
    zip_ref.extractall('./DataSet')


Mounted at /content/drive


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [7]:
def LoadDataSet(dataSetFolderPath: str):
    images = []
    newAnnotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    width, height = 416, 416
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        # cv2.imshow("image", image)
        resizedImage = cv2.resize(
            image, (width, height), interpolation=cv2.INTER_AREA)
        # cv2.imshow("resized_img", resized_img)
        images.append(resizedImage)
        x1, y1, x2, y2 = row[2], row[3], row[4], row[5]
        x1, y1 = x1 / image.shape[1], y1 / image.shape[0]
        x2, y2 = x2 / image.shape[1], y2 / image.shape[0]
        newAnnotations.append([x1, y1, x2, y2])
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique()
    del annotationsDataFrame
    X_train, X_val, y_train, y_val = train_test_split(
        images, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val, uniqueSigns



In [8]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data  # data should be a list of (input, label) pairs
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_data, label = self.data[idx]

        if self.transform:
            input_data = self.transform(input_data)
            label = self.transform(label)
        
        # Change the shape from (height, width, channels) to (channels, height, width)
        input_data = np.transpose(input_data, (2, 0, 1))
        input_data = torch.from_numpy(input_data).float()
        label = torch.tensor(label).float()
        return input_data, label

In [9]:
def _create_yolov5(input_shape, num_classes, version="s"):
    cfg = f"yolov5/models/yolov5{version}.yaml"
    model = Model(cfg, ch=3, nc=num_classes)
    return model


In [10]:
def yolov5_bbox_loss(outputs, targets):
    obj_mask, box_coords = targets[:, 0], targets[:, 1:]
    output_obj_mask, output_box_coords = outputs[:, 0], outputs[:, 1:]
    
    coord_loss = nn.MSELoss()(output_box_coords[obj_mask.bool()], box_coords[obj_mask.bool()])
    
    return coord_loss


In [19]:
# def train_epoch(model, dataloader, criterion, optimizer, device):
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         # labels = torch.stack(labels).to(device)
        
#         optimizer.zero_grad()
        
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
        
#     return running_loss / len(dataloader)

# def train_epoch(model, dataloader, optimizer, device):
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in dataloader:
#         inputs = inputs.to(device)
#         labels = labels.to(device)
        
#         optimizer.zero_grad()
        
#         outputs = model(inputs)
#         loss = yolov5_bbox_loss(outputs, labels)
#         loss.backward()
#         optimizer.step()
        
#         running_loss += loss.item()
        
#     return running_loss / len(dataloader)
def train_epoch(model, dataloader, optimizer, device, compute_loss):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss, loss_items = compute_loss(outputs, labels)  # Using the ComputeLoss instance for loss computation
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    return running_loss / len(dataloader)

In [12]:
# def validate_epoch(model, dataloader, criterion, device):
#     model.eval()
#     running_loss = 0.0
#     with torch.no_grad():
#         for inputs, labels in dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)
#             # labels = torch.stack(labels).to(device)
            
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
            
#             running_loss += loss.item()
            
#     return running_loss / len(dataloader)

# def validate_epoch(model, dataloader, device):
#     model.eval()
#     running_loss = 0.0
#     with torch.no_grad():
#         for inputs, labels in dataloader:
#             inputs = inputs.to(device)
#             labels = labels.to(device)
            
#             outputs = model(inputs)
#             loss = yolov5_bbox_loss(outputs, labels)
            
#             running_loss += loss.item()
            
#     return running_loss / len(dataloader)

def validate_epoch(model, dataloader, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = [{torch.tensor(l).float().to(device)} for l in labels]

            outputs = model(inputs)
            loss = model.loss(outputs, labels)

            running_loss += loss.item()

    return running_loss / len(dataloader)
  

In [13]:
# def train_yolov5_model(model, train_dataloader, val_dataloader, num_epochs, optimizer, device):
#     model.to(device)
#     criterion = nn.MSELoss()
    
#     for epoch in range(num_epochs):
#         start_time = time.time()
        
#         # train_loss = train_epoch(model, train_dataloader, criterion, optimizer, device)
#         # val_loss = validate_epoch(model, val_dataloader, criterion, device)
#         train_loss = train_epoch(model, train_dataloader, optimizer, device)
#         val_loss = validate_epoch(model, val_dataloader, device)
#         end_time = time.time()
#         elapsed_time = end_time - start_time
        
#         print(f"Epoch {epoch+1}/{num_epochs} - Time: {elapsed_time:.2f}s - Train Loss: {train_loss:.4f} - Val Loss: {val_loss:.4f}")
        
#     return model
def train_yolov5_model(model, train_dataloader, val_dataloader, num_epochs, optimizer, device, compute_loss):
    model.to(device)
    
    
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 10)
        
        start_time = time.time()
        
        train_loss = train_epoch(model, train_dataloader, optimizer, device, compute_loss)
        val_loss = validate_epoch(model, val_dataloader, device)
        
        print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        
        elapsed_time = time.time() - start_time
        print(f"Time elapsed: {elapsed_time:.0f}m {elapsed_time % 60:.0f}s")
    
    return model


In [14]:
X_train, X_val, y_train, y_val, uniqueSigns = LoadDataSet("./DataSet")
num_classes = len(uniqueSigns)
print(num_classes)

47


In [15]:
trainDataSet = []  # List of (input, label) pairs for the training set
valDataSet = []  # List of (input, label) pairs for the validation set
for i in range(len(X_train)):
    trainDataSet.append((X_train[i], y_train[i]))

for i in range(len(X_val)):
    valDataSet.append((X_val[i], y_val[i]))



In [16]:
trainDataSetset = CustomDataset(trainDataSet)
valDataSetset = CustomDataset(valDataSet)

# Create the data loaders
trainDataSetloader = DataLoader(
    trainDataSetset, batch_size=32, shuffle=True, num_workers=4)
valDataSetloader = DataLoader(
    valDataSetset, batch_size=32, shuffle=False, num_workers=4)
input_shape = (416, 416, 3)


In [17]:
yolov5_model = _create_yolov5(input_shape, num_classes)
optimizer = optim.Adam(yolov5_model.parameters(), lr=0.001)
compute_loss = ComputeLoss(yolov5_model)


Overriding model.yaml nc=80 with nc=47

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                
  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]            

In [20]:
num_epochs = 50
trained_model = train_yolov5_model(yolov5_model, trainDataSetloader, valDataSetloader, num_epochs, optimizer, device, compute_loss)


Epoch 1/50
----------


AttributeError: ignored

In [ ]:

# Save the trained model
torch.save(trained_model.state_dict(), 'trained_yolov5_model.pth')